# Introduction

[Instrumentation Ticket](https://phabricator.wikimedia.org/T292587)   | [QA Ticket](https://phabricator.wikimedia.org/T297852)

# Instrumentation note

Web team has deployed the instrumentation to track AB test enrollment.  
The related events will be stored in `mediawiki_web_ab_test_enrollment` schema.

In [1]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
library(tidyverse); 
library(lubridate); 
library(scales);
library(magrittr); 
library(dplyr);
})

In [2]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

# Check experiment group

__Note__:   

QAed on 12/20/2021. 

In [57]:
query <- "
select wiki, TO_DATE(dt),`group`, count(1)
FROM event.mediawiki_web_ab_test_enrollment
WHERE year=2021
GROUP BY wiki, TO_DATE(dt),`group`
"

In [58]:
df <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [59]:
df

wiki,X_c1,group,X_c3
<chr>,<chr>,<chr>,<int>
testwiki,2021-12-13,control,7
testwiki,2021-12-13,stickyHeaderEnabled,9
testwiki,2021-12-13,unsampled,8
testwiki,2021-12-14,control,163
testwiki,2021-12-14,stickyHeaderDisabled,11
testwiki,2021-12-14,stickyHeaderEnabled,85
testwiki,2021-12-14,unsampled,7
testwiki,2021-12-15,control,67
testwiki,2021-12-15,stickyHeaderDisabled,59


In [60]:
query <- "
select wiki, TO_DATE(dt) AS date_time,`group`, count(distinct web_session_id) AS sessions
FROM event.mediawiki_web_ab_test_enrollment
WHERE year=2021
GROUP BY wiki, TO_DATE(dt),`group`
"

In [61]:
df <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [62]:
df

wiki,date_time,group,sessions
<chr>,<chr>,<chr>,<int>
testwiki,2021-12-13,control,1
testwiki,2021-12-13,stickyHeaderEnabled,1
testwiki,2021-12-13,unsampled,2
testwiki,2021-12-14,control,10
testwiki,2021-12-14,stickyHeaderDisabled,3
testwiki,2021-12-14,stickyHeaderEnabled,9
testwiki,2021-12-14,unsampled,3
testwiki,2021-12-15,control,5
testwiki,2021-12-15,stickyHeaderDisabled,4


__Note:__  

Engineer confirmed that __unsampled__ sessions shouldn't be recordered.  FIXED since 12-15-2021. 

Potential bug:
Each session only need to be recorded once as it should only be assigned to one group. Currently sessions are recorded multiple times. Confirmed with engineer that it's expected. [ticket](https://phabricator.wikimedia.org/T292587#7576512)

# Check whether `mediawiki_web_ab_test_enrollment` can join with click events on stiky header

In [119]:
query <- "
SELECT distinct event.name 
FROM event.DesktopWebUIActionsTracking
WHERE event.name LIKE '%sticky%'
AND YEAR =2021
"

In [121]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [122]:
df

name
<chr>
ve-edit-protected-sticky-header
pt-userpage-sticky-header
wikitext-edit-sticky-header
ui.vector-sticky-search-form.icon
pt-mytalk-sticky-header
pt-watchlist-sticky-header
ve-edit-sticky-header
ui.dropdown-p-personal-sticky-header
utcdate-sticky-header


## Number of clicks on user menu

In [131]:
query <- 
"
WITH t1 AS (
SELECT 
 web_session_id,
 year, month, day,
 `group` AS test_group,
 min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=14
GROUP BY  web_session_id,
 year, month, day,
 `group`
)
SELECT t1.test_group, t2.event, t1.session_dt, t2.dt
FROM t1
LEFT JOIN event.DesktopWebUIActionsTracking AS t2 
ON t1.year=t2.year AND t1.month=t2.month AND t1.day=t2.day 
AND t2.event.token = t1.web_session_id
WHERE t1.year=2021 and t1.month=12 AND t1.day=14
AND t2.wiki='testwiki'
AND t2.event.name IN ('ui.dropdown-p-personal-sticky-header', 
've-edit-protected-sticky-header',
'pt-userpage-sticky-header',
'wikitext-edit-sticky-header',
'ui.vector-sticky-search-form.icon',
'pt-mytalk-sticky-header',
'pt-watchlist-sticky-header',
've-edit-sticky-header',
'utcdate-sticky-header' ,
'ui.dropdown-p-lang-btn-sticky-header' ,
'pt-mycontris-sticky-header',
'talk-sticky-header',
'history-sticky-header'
) 
LIMIT 100
"


In [132]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [139]:
query <- 
"
WITH t1 AS (
SELECT 
 web_session_id,
 year, month, day,
 `group` AS test_group,
 min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=14
GROUP BY  web_session_id,
 year, month, day,
 `group`
)
SELECT t1.test_group, sum(IF (t2.event IS NULL, 0 , 1)) AS clicks, 
COUNT(DISTINCT t2.event.token) AS sessions, COUNT(t2.event),
SUM( IF (t2.event IS NULL, 1 , 0))
FROM t1
LEFT OUTER JOIN event.DesktopWebUIActionsTracking AS t2 
ON t1.year=t2.year AND t1.month=t2.month AND t1.day=t2.day 
AND t2.event.token = t1.web_session_id
WHERE t1.year=2021 and t1.month=12 AND t1.day=14
AND t1.session_dt < t2.dt
AND t2.wiki='testwiki'
AND t2.event.name IN ('ui.dropdown-p-personal', 'pt-watchlist' , 'pt-mytalk', 
'pt-sandbox', 'pt-preferences', 'pt-betafeatures', 'pt-mycontris', 'pt-logout',
'ui.dropdown-p-personal-sticky-header', 
've-edit-protected-sticky-header',
'pt-userpage-sticky-header',
'wikitext-edit-sticky-header',
'ui.vector-sticky-search-form.icon',
'pt-mytalk-sticky-header',
'pt-watchlist-sticky-header',
've-edit-sticky-header',
'utcdate-sticky-header' ,
'ui.dropdown-p-lang-btn-sticky-header' ,
'pt-mycontris-sticky-header',
'talk-sticky-header',
'history-sticky-header'
) 
GROUP BY t1.test_group
"

In [140]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [141]:
df

test_group,clicks,sessions,X_c3,X_c4
<chr>,<int>,<int>,<int>,<int>
control,1,1,1,0
stickyHeaderDisabled,1,1,1,0
stickyHeaderEnabled,23,3,23,0
unsampled,1,1,1,0


## Number of pageview on the same day

In [68]:
query_pageview <- 
"
SELECT 
 `group` AS test_group, COUNT(1) AS pageviews, count(distinct web_session_id) AS sessions
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=19
GROUP BY `group`
"

In [69]:
df <- wmfdata::query_hive(query_pageview)

Don't forget to authenticate with Kerberos using kinit



In [70]:
df

test_group,pageviews,sessions
<chr>,<int>,<int>
control,10,4
stickyHeaderDisabled,22,3
stickyHeaderEnabled,12,4


## Check bucketing For data since 12-15-2021

In [92]:
query_pageview <- 
"
SELECT 
 `group` AS test_group, COUNT(1) AS pageviews, count(distinct web_session_id) AS sessions
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day>=15
GROUP BY `group`
"

In [93]:
df <- wmfdata::query_hive(query_pageview)

Don't forget to authenticate with Kerberos using kinit



In [94]:
df

test_group,pageviews,sessions
<chr>,<int>,<int>
control,215,31
stickyHeaderDisabled,134,23
stickyHeaderEnabled,155,26


In [102]:
total_sessions <- sum(df$sessions)

In [105]:
df <- df %>%
  mutate(session_pct=round(sessions/total_sessions * 100 , 2))

In [106]:
df

test_group,pageviews,sessions,session_pct
<chr>,<int>,<int>,<dbl>
control,215,31,38.75
stickyHeaderDisabled,134,23,28.75
stickyHeaderEnabled,155,26,32.50


__Note:__ 

The distribution ratio of control, stickyHeaderDisabled and stickyHeaderEnabled groups is close to 1:1:1

# Check whether `mediawiki_web_ab_test_enrollment` can join with scroll events

In [83]:

query <- 
"
WITH t1 AS (
SELECT 
 web_session_id,
 year, month, day,
 `group` AS test_group,
 min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=19
GROUP BY  web_session_id,
 year, month, day,
 `group`
)
SELECT t1.test_group, t2.meta, t1.session_dt, t2.dt
FROM t1
LEFT JOIN event.mediawiki_web_ui_scroll AS t2 
ON t1.year=t2.year AND t1.month=t2.month AND t1.day=t2.day 
AND t2.web_session_id = t1.web_session_id
WHERE t1.year=2021 and t1.month=12 AND t1.day=19
AND t2.meta.domain='test.wikipedia.org'
LIMIT 100
"

In [84]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [142]:

query_scrolls <- 
"
WITH t1 AS (
SELECT 
 web_session_id,
 year, month, day,
 `group` AS test_group,
 min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=14
GROUP BY  web_session_id,
 year, month, day,
 `group`
)
SELECT t1.test_group,sum(IF (t2.meta IS NULL, 0 , 1)) AS scrolls
FROM t1
LEFT JOIN event.mediawiki_web_ui_scroll AS t2 
ON t1.year=t2.year AND t1.month=t2.month AND t1.day=t2.day 
AND t2.web_session_id = t1.web_session_id
WHERE t1.year=2021 and t1.month=12 AND t1.day=14
AND t1.session_dt < t2.dt
AND t2.meta.domain='test.wikipedia.org'
GROUP BY t1.test_group
"

In [143]:
df_scrolls <- wmfdata::query_hive(query_scrolls)

Don't forget to authenticate with Kerberos using kinit



In [144]:
df_scrolls

test_group,scrolls
<chr>,<int>
control,17
stickyHeaderEnabled,28
unsampled,1


In [148]:
query_scrolls <- 
"
WITH t1 AS (
SELECT 
 web_session_id,
 year, month, day,
 `group` AS test_group,
 min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki='testwiki' and year=2021 and month=12 and day=28
GROUP BY  web_session_id,
 year, month, day,
 `group`
)
SELECT t1.test_group,sum(IF (t2.meta IS NULL, 0 , 1)) AS scrolls
FROM t1
LEFT JOIN event.mediawiki_web_ui_scroll AS t2 
ON t1.year=t2.year AND t1.month=t2.month AND t1.day=t2.day 
AND t2.web_session_id = t1.web_session_id
WHERE t1.year=2021 and t1.month=12 AND t1.day=28
AND t1.session_dt < t2.dt
AND t2.meta.domain='test.wikipedia.org'
GROUP BY t1.test_group
"

In [149]:
df_scrolls <- wmfdata::query_hive(query_scrolls)

Don't forget to authenticate with Kerberos using kinit



In [150]:
df_scrolls

test_group,scrolls
<chr>,<int>
control,36
stickyHeaderDisabled,3


__Note:__

We only see scroll-back event on `stickyHeaderDisabled` group, not in `stickyHeaderEnabled` group. Maybe it indicates that sticky header reduced scroll-back on 12-28-2021. We needs more data to confirm after we collect data from  AB test 